In [122]:
# 导入数据
# 数据预处理
# 构建G（生成者）和D（判别者）网络
# 训练过程
# 保存模型
# 测试

In [123]:
import time

import tensorflow as tf
from tensorflow.keras import layers
import numpy as np
import matplotlib.pyplot as plt

In [125]:
tf.config.list_physical_devices('GPU')

[]

In [103]:
# 导入数据
(train_imgs, train_labels), (test_imgs, test_labels) = tf.keras.datasets.mnist.load_data()
train_imgs.shape

(60000, 28, 28)

In [104]:
#增加维度
train_imgs = train_imgs.reshape(-1, 28, 28, 1).astype("float32")

In [105]:
# 像素值调整到 【1 - 1】
train_imgs = (train_imgs - 127.5) / 127.5
train_imgs.max(), train_imgs.min()

(1.0, -1.0)

In [106]:
# 构建dataloader
batch_size = 128
# shuffle把样本打乱（传入样本数量）
train_imgs = tf.data.Dataset.from_tensor_slices(train_imgs).shuffle(60000).batch(batch_size)

In [107]:
# 获取一张图片显示

for img in train_imgs:
    plt.imshow(img[0], cmap='gray')
    break

<img src="model_arc.png" width = "1200"/>

In [108]:
#构建网络
G_Model = tf.keras.Sequential([
    # 将长度为100的噪声向量 output = 7*7*256
    layers.Dense(7 * 7 * 256, use_bias=False, input_shape=(100,)),
    #BN
    layers.BatchNormalization(),
    #ReLU
    layers.LeakyReLU(),
    #reshape
    layers.Reshape((7, 7, 256)),

    #上采样
    # input：7*7*256
    # output：7*7*128
    layers.Conv2DTranspose(128, 5, 1, use_bias=False, padding="same"),  # 转置卷积上采样
    #BN
    layers.BatchNormalization(),
    # ReLU
    layers.LeakyReLU(),

    # 上采样
    #input：7*7*128
    #output：14*14*64
    layers.Conv2DTranspose(64, 5, 2, use_bias=False, padding="same"),

    #BN
    layers.BatchNormalization(),
    # ReLU
    layers.LeakyReLU(),

    # 上采样
    # inout：14*14*64
    #output：28*28*1
    layers.Conv2DTranspose(1, 5, 2, use_bias=False, padding="same", activation="tanh")

])

In [109]:
G_Model.summary()

Model: "sequential_8"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_8 (Dense)              (None, 12544)             1254400   
_________________________________________________________________
batch_normalization_20 (Batc (None, 12544)             50176     
_________________________________________________________________
leaky_re_lu_20 (LeakyReLU)   (None, 12544)             0         
_________________________________________________________________
reshape_4 (Reshape)          (None, 7, 7, 256)         0         
_________________________________________________________________
conv2d_transpose_12 (Conv2DT (None, 7, 7, 128)         819200    
_________________________________________________________________
batch_normalization_21 (Batc (None, 7, 7, 128)         512       
_________________________________________________________________
leaky_re_lu_21 (LeakyReLU)   (None, 7, 7, 128)        

In [110]:
# 随机产生噪声向量G
noise_V = tf.random.normal([1, 100])
noise_V

<tf.Tensor: shape=(1, 100), dtype=float32, numpy=
array([[ 0.43493873,  0.89592373,  1.3176631 ,  0.19753818, -0.33090314,
         0.6812213 , -0.65661466,  1.0094072 ,  1.017605  , -1.1575909 ,
        -0.47496653,  1.9417968 , -2.2622354 , -0.32122764, -0.96700954,
         0.17363095, -1.2748486 , -0.16147394,  0.66193146,  0.5423956 ,
        -0.37913677, -0.05083169,  1.4106808 , -0.2803393 , -1.0569738 ,
        -0.00950678, -0.29995933, -1.0769626 , -0.93890035,  0.11588674,
         1.228505  , -1.7195743 ,  0.62880635,  1.5324222 ,  0.8717593 ,
        -0.36493847,  0.7215779 , -1.077011  ,  0.5422991 ,  0.3187023 ,
         1.8156302 , -1.9048982 , -1.6527851 ,  0.5951544 , -0.9584329 ,
         0.68829733,  1.7350827 , -0.40268284, -0.6127545 ,  0.55791557,
        -0.24849086, -0.7971608 ,  0.9247298 ,  1.3809065 ,  0.3416402 ,
         1.5966105 , -0.47945967,  1.046865  ,  0.3152291 ,  1.1672324 ,
         1.2575252 , -0.8613774 ,  1.5653471 ,  0.62275934,  1.0558511 ,
 

In [111]:
generated_img = G_Model(noise_V, training=False)

In [112]:
plt.imshow(generated_img[0], cmap='gray')

In [113]:
# 构建判别器D
D_Model = tf.keras.Sequential([
    #input:28*28*1
    #output:14*14*64
    layers.Conv2D(64, 5, 2, padding="same", input_shape=(28, 28, 1)),
    #BN
    layers.BatchNormalization(),
    #ReLU
    layers.LeakyReLU(),
    #DP
    layers.Dropout(0.3),

    #卷积 input：14*14**64
    #output：7*7*128
    layers.Conv2D(128, 5, 2, padding="same"),

    #BN
    layers.BatchNormalization(),
    #ReLU
    layers.LeakyReLU(),
    # DP
    layers.Dropout(0.3),

    # 拉长
    layers.Flatten(),
    #input: 6272
    #output:1
    layers.Dense(1)
])

In [114]:
D_Model.summary()

Model: "sequential_9"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_8 (Conv2D)            (None, 14, 14, 64)        1664      
_________________________________________________________________
batch_normalization_23 (Batc (None, 14, 14, 64)        256       
_________________________________________________________________
leaky_re_lu_23 (LeakyReLU)   (None, 14, 14, 64)        0         
_________________________________________________________________
dropout_8 (Dropout)          (None, 14, 14, 64)        0         
_________________________________________________________________
conv2d_9 (Conv2D)            (None, 7, 7, 128)         204928    
_________________________________________________________________
batch_normalization_24 (Batc (None, 7, 7, 128)         512       
_________________________________________________________________
leaky_re_lu_24 (LeakyReLU)   (None, 7, 7, 128)        

In [115]:
# 将刚才生成的G图片输入D
output = D_Model(generated_img, training=False)

In [116]:
output

<tf.Tensor: shape=(1, 1), dtype=float32, numpy=array([[0.00261324]], dtype=float32)>

In [117]:
# 定义损失函数和优化器
cross_entropy = tf.keras.losses.BinaryCrossentropy(from_logits=True)

G_optimizer = tf.keras.optimizers.Adam(0.0001)
D_optimizer = tf.keras.optimizers.Adam(0.0001)

In [118]:
# 训练判别网络D
# 真实图片识别为1
# 伪造图片识别为0

# 训练生成网络
# 伪造图片，使得判别网络D识别为1


In [119]:
@tf.function
def train_step(imgs):
    # 训练判别网络:真实图片判别为1
    with tf.GradientTape() as tape:  # 梯度计算
        # 真实图片的判别输出
        real_output = D_Model(imgs, training=True),
        # 构造全1 labels
        ones_label = tf.ones_like(real_output)
        # 计算loss
        d_loss = cross_entropy(ones_label, real_output)

    #计算梯度
    d_gradient = tape.gradient(d_loss, D_Model.trainable_variables)
    #优化器优化
    D_optimizer.apply_gradients(zip(d_gradient, D_Model.trainable_variables))

    # 伪造图片识别为0
    with tf.GradientTape() as tape:
        # 伪造图片
        # 构建一个噪声向量
        noise_vector = tf.random.normal([batch_size,100])
        # 生成的图片
        fake_img = G_Model(noise_vector,training = True)

        # 伪造图片的判别输出
        fake_output = D_Model(fake_img,training = True)
        # 构建全0 labels
        zeros_label = tf.zeros_like(fake_output)

        # 计算loss
        d_loss = cross_entropy(zeros_label, fake_output)

    # 计算梯度
    d_gradient = tape.gradient(d_loss,D_Model.trainable_variables)
    # 优化器
    D_optimizer.apply_gradients(zip(d_gradient,D_Model.trainable_variables))


    # ---------------
    # 训练生成网络：伪造图片，使得判别网络D识别为1
    with tf.GradientTape() as tape:
        # 构建一个噪声向量
        noise_vector = tf.random.normal((batch_size,100))
        # 生成的图片
        fake_img = G_Model(noise_vector,training = True)

        # 伪造图片的判别输出
        fake_output = D_Model(fake_img,training = True)
        # 构建全1 labels(希望判别器把伪造的图片识别为1)
        ones_label = tf.ones_like(fake_output)

        # 计算loss
        g_loss = cross_entropy(ones_label, fake_output)

    # 计算梯度
    g_gradient = tape.gradient(g_loss,G_Model.trainable_variables)

    # 优化器优化
    G_optimizer.apply_gradients(zip(g_gradient,G_Model.trainable_variables))





In [120]:
test_noise = tf.random.normal([16,100])  # 测试给16张图片即可，维度为100

In [121]:
# 开始训练
import time
epoch_num = 100
for epoch in range(epoch_num):
    # 获取批次图片(128张图片)
    # 计算训练一个epoch的实践
    startT = time.time()
    for img_batch in train_imgs:
        # 训练
        train_step(img_batch)

    print("第{}个epoch执行完毕，耗时{}s，将显示当前G网络的生成图片".format(epoch+1,time.time() - startT))

    # 测试当前网络
    generated_img = G_Model(test_noise,training = False)
    # 显示这16张测试图片
    fig = plt.figure(figsize=(4,4))
    for i in range(16):
        plt.subplot(4,4,i + 1)
        plt.imshow(generated_img[i] * 127.5 + 127.5,cmap = 'gray')
        plt.axis('off')  # 不显示轴
    plt.show()

第1个epoch执行完毕，耗时183.1152994632721s，将显示当前G网络的生成图片


<IPython.core.display.Javascript object>

第2个epoch执行完毕，耗时180.30792093276978s，将显示当前G网络的生成图片


<IPython.core.display.Javascript object>

Exception ignored in: <function IteratorResourceDeleter.__del__ at 0x0000021F74BDD550>
Traceback (most recent call last):
  File "E:\Anaconda3\envs\tf\lib\site-packages\tensorflow\python\data\ops\iterator_ops.py", line 537, in __del__
  File "E:\Anaconda3\envs\tf\lib\site-packages\tensorflow\python\ops\gen_dataset_ops.py", line 1137, in delete_iterator
KeyboardInterrupt: 


KeyboardInterrupt: 